Terms
`Large Langauge Models (LLMS)` - AI models specificlaly trained to understand and generate human language.
`Variational Autoencoders (VAE)` - AI models that can be used to create new images. Made up of 2 parts: The **encoder** reduces data to a simpler form, and the **decoder** expands it back to generate new content.
`Parameters` - The varaibles that th emdoel learns during training. They are internal to the model and are adjusted through the learning process. In Neural Networks, they typically include weights and biases.
`Weights` - coefficients for the input data. Used in calclulations to dtermine the importance or influcence of input variables to the model's output. In a NN, each connection between neurons has an associated weight.
`Biases` - Additional constants attached to neurons and are added to the weighted input before the activation funciton is applied. Biases ensure a non-zero output when all in the inputs are zero.
`Hyperparameters` - Unlike Parameters, theese are not learned from the data, but are more like settings or configurations for the learning process. THey're set prior to the training and remain constant throughout. External to the model and used to control training

Subword Tokenization is breaking down words into smalelr unites, such as parts of words or whole words.
`tokenizer` - Tokenizes text
`AutmodelForAausalLM.from_pretrained` - Loads a pretrained model
`AutoTokenizer.from_pretrained` - Loads a tokenizer
`torch.nn.functional.softmax` - Converts logits into probabilites

In [ ]:
# Load a tokenizer and model
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load a pretrained model and tokenizer using Huggingface
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Create a partial sentence and tokenize it
text = "Following examples is the best way to learn about generative"
inputs = tokenizer(text, return_tensors="pt")

# showw the tokenns as numbers (their ID)
inputs["input_ids"]

# output: tensor([[  52,   67, 4355,  318,  262, 1266, 1295,  284, 2193,  546, 1152,  876]])

Examining the tokens - what do they mean?

In [ ]:
import pandas as pd;

def show_tokenizations(inputs):
    return pd.DataFrame(
        [(id, tokenizer.decode(id)) for id in inputs["input_ids"][0]],
        columns = ["id", "token"]
    )

show_tokenizations(inputs)

2. Calculate the probablitly of the next token
- Tokens are not just letters nor words, they can represent words, parts of words, or punctionations. This is called *subword tokenization*. 

In [ ]:
#Calcuate the probabilities for the next token for all possible choices. Show the top 5 and the corresponding words or subwords for those 5 tokens.
import torch

with torch.no_grad(): # no_grad() means this will not be used for training
    logits = model(**inputs).logits[:,-1,:]
    probabilities = torch.nn.functional.softmax(logits[0], dim=-1)

def show_next_token_choices(probabilities, top_n=5):
    return pd.DataFrame(
        [
            (id, tokenizer.decode(id), p.item())
            for id, p in enumerate(probabilities)
            if p.item()
        ],
        columns =["id","token","p"],
    ).sort_values("p", ascending=False)[:top_n]

show_next_token_choices(probabilities)

In [ ]:
# Obtain the token id for the most probable next token
next_token_id= torch.argmax(probabilities).item()
print(f"next token id: {next_token_id}")
print(f"Next token: {tokenizer.decode(next_token_id)}")

# Append the most likely next token to the original text
text = text + tokenizer.decode(8300) #8300 was the id of most likely token

In [ ]:
from IPython.display import Markdown, display

# Show the original text
print(text)

# Convert to tokens
inputs = tokenizer(text, return_tensors="pt")

# Calculate the probabilities of the next token and show the top 5 choices
with torch.no_grad():
    logits = model(**inputs).logits[:, -1, :]
    probabilities = torch.nn.functional.softmax(logits[0], dim=0)

display(Markdown("**Next token probabilities:**"))
display(show_next_token_choices(probabilities))

# Choose the most likely token ID and add it to the text
next_token_id = torch.argmax(probabilities).item()
text = text + tokenizer.decode(next_token_id)

In [ ]:
# Use the Generate method
from IPython.display import Markdown,display

#start with some text and tokenize it
text = "Once upon a time, generative models"
inputs = tokenizer(text, return_tensors="pt")

# Use the 'generate' method to generate a bunch of text
## This causes kernel crash locally, so not run
output = model.generate(**inputs, max_length=100, pad_token_id=tokenizer.eos_token_id)

# Show the generated text
display(Markdown(tokenizer.decode(output[0])))